In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import time
import re
import unidecode
import gc

In [ ]:
data = pd.read_csv('~/micromamba/UNC_Abierta_Predict/data-1740750271190-nueva.csv' , low_memory  = False)

In [ ]:
data.head()

Esta función primero revisa los NaN en las direcciones y reemplaza por una cadena vacia. Luego aplica toupper, remueve acentos y puntos.

In [ ]:

def clean_address_component(text):
    """Limpiamos nombres"""
    if pd.isna(text):
        return ""
    text = str(text).upper().strip()
    text = unidecode.unidecode(text)  # saca acentos
    text = re.sub(r'[^\w\s]', '', text)  # saca puntos
    return text

Estandarizamos direcciones para que Nominatim las reconozca. Primero se cambian las abrevaciones del tipo "AV" por "AVENIDA". Luego se limpian de nuevo los nombres de las calles; todo lo que sea un vacío o valor NaN lo establece como "None". For pattern aplica todos esos cambios.

Posteriormente se agrega el numero de cada direccion al nombre de la calle en una misma variable, fijando Cordoba Argentina (nuestra región de análisis) para todos los casos.

Address une toda la información en un mismo string y genera la columna "ubicacion".

In [ ]:

def format_full_address(row):
    """Lleva a formato de direcciones de Nominatim"""
    replacements = {
        r'\bAV\b': 'AVENIDA',
        r'\bBV\b': 'BOULEVARD',
        r'\bSN\b': 'S/N',
        r'\bDTO\b': 'DEPARTAMENTO',
        r'\bPJE\b': 'PASAJE',
        r'\bESQ\b': '&',
        r'\bESQUINA\b': '&',
        r'\bNAN\b': ''
    }

    street = clean_address_component(row['calle'])
    if not street:
        return None

    for pattern, replacement in replacements.items():
        street = re.sub(pattern, replacement, street)

    numero = row['numero']
    if pd.notna(numero):
        try:
            street += f" {int(numero)}"
        except:
            street += " S/N"
    else:
        street += " S/N"

    components = [
        street,
        clean_address_component(row['barrio']),
        clean_address_component(row['localidad']),
        "Córdoba",
        "Argentina"
    ]

    address = ', '.join(filter(None, [c.strip() for c in components if c.strip()]))
    return address if len(address) > 10 else None

In [ ]:
data['ubicacion'] = data.apply(format_full_address, axis=1)

In [ ]:
data_ubi = data[["ubicacion", "persona"]].dropna(subset=['ubicacion'])
data_ubi = data_ubi[data_ubi['ubicacion'].str.len() > 10]
print(data_ubi.head())

El servicio Nominatim de OSM solicita login con un nombre de proyecto y un usuario/mail. También  limita la cantidad de iteraciones por segundo, por lo que no puede paralelizarze por núcleos de forma remota. Es necesario instalar un Docker local (más información en el .txt adjunto).

geolocator.geocode busca las direcciones de "ubicacion" y si las encuentra le asigna las geometrias lat, lon; en caso contrario, devuelve "None None".

for itera el proceso y cada 100 filas nos devuelve el avance, con un time sleep de 1.5; al finalizar, se guarda el resultado en un archivo .gpkg, que contendrá puntos para todas las filas (ubicaciones o direcciones) que hayamos procesado.

In [ ]:
# Geocoder de Nominatim
from concurrent.futures import ThreadPoolExecutor, as_completed
from shapely.geometry import Point

geolocator = Nominatim(
    user_agent="unc_abierta_CCAD (stefano.balbo@mi.unc.edu.ar)",
    timeout=20,
    domain="localhost:8080",
    scheme="http"
)

def geocode_address(ubicacion):
    try:
        location = geolocator.geocode(ubicacion, exactly_one=True, addressdetails=True)
        return (location.latitude, location.longitude) if location else (None, None)
    except Exception as e:
        print(f"Error en {ubicacion[:30]}...: {str(e)[:100]}")
        return (None, None)

def batch_geocode(addresses, batch_size=1000, max_workers=120):
    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(geocode_address, addr): addr for addr in addresses}

        for idx, future in enumerate(as_completed(futures)):
            addr = futures[future]
            results[addr] = future.result()
            if idx % 1000 == 0:
                print(f"Procesados: {idx}/{len(addresses)} ({idx/len(addresses):.1%})")
    return results

address_list = data_ubi['ubicacion'].tolist()

geocode_results = batch_geocode(address_list, max_workers=120)

data_ubi['coords'] = data_ubi['ubicacion'].map(geocode_results)
data_ubi[['lat', 'lon']] = pd.DataFrame(data_ubi['coords'].tolist(), index=data_ubi.index)

data_ubi_clean = data_ubi.dropna(subset=['lat', 'lon']).copy()
geometry = [Point(lon, lat) for lon, lat in zip(data_ubi_clean.lon, data_ubi_clean.lat)]

gdf = gpd.GeoDataFrame(
    data_ubi_clean,
    geometry=geometry,
    crs="EPSG:4326"
)

gdf.to_file(
    "geocoded_results.gpkg",
    driver="GPKG",
    layer='estudiantes',
    encoding='utf-8',
    index=False
)

Procesados: 0/545532 (0.0%)
Procesados: 1000/545532 (0.2%)
Procesados: 2000/545532 (0.4%)
Procesados: 3000/545532 (0.5%)
Procesados: 4000/545532 (0.7%)
Procesados: 5000/545532 (0.9%)
Procesados: 6000/545532 (1.1%)
Procesados: 7000/545532 (1.3%)
Procesados: 8000/545532 (1.5%)
Procesados: 9000/545532 (1.6%)
Procesados: 10000/545532 (1.8%)
Procesados: 11000/545532 (2.0%)
Procesados: 12000/545532 (2.2%)
Procesados: 13000/545532 (2.4%)
Procesados: 14000/545532 (2.6%)
Procesados: 15000/545532 (2.7%)
Procesados: 16000/545532 (2.9%)
Procesados: 17000/545532 (3.1%)
Procesados: 18000/545532 (3.3%)
Procesados: 19000/545532 (3.5%)
Procesados: 20000/545532 (3.7%)
Procesados: 21000/545532 (3.8%)
Procesados: 22000/545532 (4.0%)
Procesados: 23000/545532 (4.2%)
Procesados: 24000/545532 (4.4%)
Procesados: 25000/545532 (4.6%)
Procesados: 26000/545532 (4.8%)
Procesados: 27000/545532 (4.9%)
Procesados: 28000/545532 (5.1%)
Procesados: 29000/545532 (5.3%)
Procesados: 30000/545532 (5.5%)
Procesados: 31000/545